### An illustration of SIMD packing in BGV/BFV FHE schemes for integer vector arithmetic

In [ ]:
import sympy
from sympy import symbols, Poly, GF

# ==========================================
# 1. SETUP THE RING PARAMETERS
# ==========================================
m = 8
p = 73
n = m // 2

x = symbols('x')
# Define the ring modulus polynomial: x^n + 1
ring_modulus = Poly(x**n + 1, x, domain=GF(p))

# Find the "Slots" (roots of unity) in a very lazy way!
# We calculate strictly in integer arithmetic
roots = [r for r in range(p) if (pow(r, n, p) + 1) % p == 0]

print("-" * 50)
print(f"Setup: Ring Z_{p}[x] / (x^{n} + 1)")
print(f"Found {len(roots)} slots (roots) at indices: {roots}")

--------------------------------------------------
Setup: Ring Z_73[x] / (x^4 + 1)
Found 4 slots (roots) at indices: [10, 22, 51, 63]


In [49]:
# ==========================================
# HELPER: INVERSE CRT / LAGRANGE INTERPOLATION IN Z_p
# ==========================================
def inverse_crt_interpolate(points, variable, modulus):
    """
    Computes the Lagrange Polynomial strictly in GF(p).
    Given points [(x0, y0), (x1, y1)...], find P(x) s.t. P(xi) = yi.
    
    Formula: P(x) = sum( yi * Li(x) )
    Where Li(x) = prod( (x - xj) * (xi - xj)^-1 ) for j != i
    """
    final_poly = Poly(0, variable, domain=GF(modulus))
    
    # Extract x and y coordinates
    xs = [pt[0] for pt in points]
    ys = [pt[1] for pt in points]
    k = len(points)
    
    for i in range(k):
        xi, yi = xs[i], ys[i]
        
        # Start building the basis polynomial Li(x)
        # We start with numerator = 1, denominator = 1
        li_numerator = Poly(1, variable, domain=GF(modulus))
        li_denominator = 1
        
        for j in range(k):
            if i == j:
                continue
            
            xj = xs[j]
            
            # Numerator term: (x - xj)
            term_poly = Poly(x - xj, variable, domain=GF(modulus))
            li_numerator = li_numerator * term_poly
            
            # Denominator term: (xi - xj)
            # We calculate this as a scalar integer modulo p
            denom_term = (xi - xj) % modulus
            li_denominator = (li_denominator * denom_term) % modulus
            
        # Calculate modular inverse of the denominator
        # standard python pow(a, -1, p) gives modular inverse
        denom_inv = pow(li_denominator, -1, modulus)
        
        # Combine: yi * numerator * denom_inv
        term = li_numerator.mul_ground((yi * denom_inv) % modulus)
        
        final_poly = final_poly + term
        
    return final_poly


In [50]:
# ==========================================
# 2. DEFINE INPUT DATA
# ==========================================
# Ensure inputs are modulo p to start with (handles negative numbers)
u_raw = [0, 1, 2, 3]
v_raw = [4, 5, 6, 7]

u = [val % p for val in u_raw]
v = [val % p for val in v_raw]

print("-" * 50)
print("u (mod p): ", u)
print("v (mod p): ", v)


--------------------------------------------------
u (mod p):  [0, 1, 2, 3]
v (mod p):  [4, 5, 6, 7]


In [51]:
# ==========================================
# 3. PACKING (ENCODING)
# ==========================================
points_u = list(zip(roots, u))
points_v = list(zip(roots, v))

# Perform Interpolation using our custom Z_p function
packed_poly_U = inverse_crt_interpolate(points_u, x, p)
packed_poly_V = inverse_crt_interpolate(points_v, x, p)

print("-" * 50)
print("PACKING STEP")
print("Polynomial U(x):")
print(packed_poly_U.as_expr())
print("\nPolynomial V(x):")
print(packed_poly_V.as_expr())

--------------------------------------------------
PACKING STEP
Polynomial U(x):
13*x**3 - 19*x - 35

Polynomial V(x):
13*x**3 - 19*x - 31


In [52]:
# ==========================================
# 4. SIMD OPERATION CHECK
# ==========================================
# Addition: Element-wise addition in the ring
sum_poly = (packed_poly_U + packed_poly_V) % ring_modulus

# Multiplication: Element-wise multiplication in the ring
# IMPORTANT: In the ring Z_p[x] / (x^n + 1), multiplication increases degree.
# We must reduce the result modulo (x^n + 1).
prod_poly = (packed_poly_U * packed_poly_V) % ring_modulus

print("-" * 50)
print("Polynomial Sum(x):")
print(sum_poly.as_expr())
print("\nPolynomial Prod(x):")
print(prod_poly.as_expr())

--------------------------------------------------
Polynomial Sum(x):
26*x**3 + 35*x + 7

Polynomial Prod(x):
18*x**3 - 27*x**2 + 13*x - 27


In [53]:
# ==========================================
# 5. UNPACKING (DECODING)
# ==========================================
# Evaluate at roots.
# .eval() returns a GF element, we cast to int for display.
recovered_data_U = [int(packed_poly_U.eval(r)) for r in roots]
recovered_data_V = [int(packed_poly_V.eval(r)) for r in roots]
sum_data = [int(sum_poly.eval(r)) for r in roots]
prod_data = [int(prod_poly.eval(r)) for r in roots]

print("-" * 50)
print("UNPACKING STEP")

# Check Inputs
is_match_u = (recovered_data_U == u)
print(f"Recovered U matches input? {is_match_u}")

is_match_v = (recovered_data_V == v)
print(f"Recovered V matches input? {is_match_v}")

# Check Sum
print("-" * 50)
print("SIMD CHECK (U + V)")
target_sum = [(i + j) % p for i, j in zip(u, v)]
print(f"Expected Sum: {target_sum}")
print(f"Actual Sum:   {sum_data}")
print(f"Match?        {sum_data == target_sum}")

# Check Product
print("-" * 50)
print("SIMD CHECK (U * V)")
target_prod = [(i * j) % p for i, j in zip(u, v)]
print(f"Expected Prod: {target_prod}")
print(f"Actual Prod:   {prod_data}")
print(f"Match?         {prod_data == target_prod}")

--------------------------------------------------
UNPACKING STEP
Recovered U matches input? True
Recovered V matches input? True
--------------------------------------------------
SIMD CHECK (U + V)
Expected Sum: [4, 6, 8, 10]
Actual Sum:   [4, 6, 8, 10]
Match?        True
--------------------------------------------------
SIMD CHECK (U * V)
Expected Prod: [0, 5, 12, 21]
Actual Prod:   [0, 5, 12, 21]
Match?         True
